### <center> This notebook has the postprocess of raw Level 3 ASCAT/QuikSCAT data and the following construction of a surface wind dataset for the tropical band and the coastal bands of the south east and north east pacific.


The data used will be the full global grid of the ASCAT/QuikSCAT dataset. The idea is to load the data into chunks and then extract only the tropical band and the whole east pacific. With that data on hand a hovmoller will be built from the meridional average of the tropical dataset and the zonal average of the coastal dataset. The variables to be used will be the meridional and zonal componentes of the 10 meter winds (not the stress, since S2S gives only 10m wind forecast). In addition for the coastal bands the alongshore wind will be computed using the land/sea mask. 

In [1]:
# Imports
import xarray as xr
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

# Just for supressing an annoying warning
import dask
dask.config.set({"array.slicing.split_large_chunks": False})

In [9]:
# Load ASCAT/QuikSCAT masks
mask = xr.open_dataset('data/ASCAT_masks.nc')

In [47]:
# Load scatterometer data

# ASCAT
path = [glob(f'data/ASCAT/{yr}*') for yr in range(2008,2023+1)]
path = sorted(sum(path,[]))

def preprocess(data):
    print(data.time.dt.strftime('%F').data)
    data = data.rename({'latitude':'lat','longitude':'lon'}).squeeze().drop('depth')
    data = data[['wind_stress',
                 'surface_downward_eastward_stress',
                 'surface_downward_northward_stress']]
    data = data.rename({'wind_stress':'tau',
                 'surface_downward_eastward_stress':'taux',
                 'surface_downward_northward_stress':'tauy'})
    data = data.sel(lat=slice(-45,45))
    data.coords['lon'] = xr.where(data.lon<0,data.lon+360,data.lon)
    data = data.sortby('lon').sel(lon=slice(110,291))
    return data

scat = xr.open_mfdataset(path, preprocess=preprocess, parallel=True, concat_dim='time', combine='nested')
    
# scat = []
# for p in path:
#     print(p)
#     data = xr.open_dataset(p).rename({'latitude':'lat','longitude':'lon'}).squeeze().drop('depth')
#     data = data[['wind_stress',
#                  'surface_downward_eastward_stress',
#                  'surface_downward_northward_stress']]
#     data = data.rename({'wind_stress':'tau',
#                  'surface_downward_eastward_stress':'taux',
#                  'surface_downward_northward_stress':'tauy'})
#     data = data.sel(lat=slice(-45,45))
#     data.coords['lon'] = xr.where(data.lon<0,data.lon+360,data.lon)
#     data = data.sortby('lon').sel(lon=slice(110,291))
#     scat.append(data)
# print('join')
# scat = xr.concat(scat,'time')

['2010-10-08']
['2009-02-21']
['2009-07-15']
['2011-01-23']
['2013-09-28']
['2016-01-23']
['2010-10-18']
['2022-12-26']
['2014-10-16']
['2013-09-02']
['2013-09-14']
['2021-05-12']
['2010-01-08']
['2008-07-01']
['2013-10-26']
['2011-10-16']
['2013-04-27']
['2016-07-12']
['2011-08-29']
['2020-12-30']
['2020-02-19']
['2014-11-13']
['2009-02-07']
['2013-01-06']
['2014-05-16']
['2017-06-04']
['2018-07-07']
['2011-07-05']
['2015-05-29']
['2020-12-29']
['2017-06-27']
['2015-06-20']
['2021-09-10']
['2018-05-09']
['2020-10-21']
['2020-08-12']
['2008-09-14']
['2018-08-13']
['2009-03-25']
['2018-10-08']
['2019-11-28']
['2012-10-21']
['2015-07-07']
['2022-12-12']
['2012-04-24']
['2018-03-10']
['2009-11-19']
['2018-07-23']
['2019-02-12']
['2017-01-19']
['2019-09-14']
['2022-06-12']
['2017-01-11']
['2014-12-02']
['2016-08-06']
['2011-11-24']
['2013-01-24']
['2023-03-17']
['2022-08-11']
['2010-09-01']
['2010-08-27']
['2010-09-24']
['2012-12-20']
['2014-12-03']
['2023-03-16']
['2020-12-21']
['2008-03-